In [ ]:
#DATASET 
!wget http://cb.lk/covid_19

In [ ]:
!unzip covid_19

In [ ]:
#CODE
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [ ]:
# CNN Based Model
# Its a sequential model, which means it has layers
# Each layer has certain filters. like layer1 has n no of filters, layer2 has m no of filters and so on.

# Here we will be creating 3 or 4 cnn layers followed by classification layers

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))

#First of all we added a conv layer with 32 filters, we're keeping no of filters small in the beginning
# lower layers detect features in v small part of image, if u just see a v small part of an img then u will
# make a v few patterns.
# But as you go deep into the network, ur receptive field of cnn layer increases ie the features it extract are 
# are based upon a bigger part of the original image. **CONCEPT OF RECEPTIVE FIELD**


model.add(Conv2D(64, (3, 3), activation='relu'))    # add another conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))           # add maxpooling layer
model.add(Dropout(0.25))
# we havent used vgg bcs it will overfit. watch tut at 57.00

# SECOND conv layer
model.add(Conv2D(64, (3, 3), activation='relu'))    # add another conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))           # add maxpooling layer
model.add(Dropout(0.25))

# THIRD conv layer
model.add(Conv2D(128, (3, 3), activation='relu'))    # add another conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))           # add maxpooling layer
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])


model.summary()


In [ ]:
# Train from scratch
# make the traing_data objects
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True, 
)

test_dataset = image.ImageDataGenerator(rescale=1./255)


In [ ]:

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

In [ ]:
train_generator.class_indices

In [ ]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

In [ ]:
hist = model.fit(
    train_generator,
    steps_per_epoch=6,
    epochs = 8,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/8
6/6 [==============================] - 60s 10s/step - loss: 2.0581 - accuracy: 0.5080 - val_loss: 0.6972 - val_accuracy: 0.5000
Epoch 2/8
6/6 [==============================] - 57s 10s/step - loss: 0.7109 - accuracy: 0.5162 - val_loss: 0.6876 - val_accuracy: 0.8333
Epoch 3/8
6/6 [==============================] - 59s 10s/step - loss: 0.6329 - accuracy: 0.7510 - val_loss: 0.5931 - val_accuracy: 0.8667
Epoch 4/8
6/6 [==============================] - 57s 9s/step - loss: 0.5475 - accuracy: 0.6945 - val_loss: 0.3831 - val_accuracy: 0.9167
Epoch 5/8
6/6 [==============================] - 62s 11s/step - loss: 0.3913 - accuracy: 0.8287 - val_loss: 0.2950 - val_accuracy: 0.9167
Epoch 6/8
6/6 [==============================] - 57s 9s/step - loss: 0.2696 - accuracy: 0.9060 - val_loss: 0.1459 - val_accuracy: 0.9500
Epoch 7/8
6/6 [==============================] - 57s 9s/step - loss: 0.2424 - accuracy: 0.8942 - val_loss: 0.1175 - val_accuracy: 0.9333
Epoch 8/8
6/6 [======================

In [ ]:
score = model.evaluate(validation_generator, verbose=0)
print('loss: ', score[0])
print('accuracy: ', score[1])

loss:  0.07823485881090164
accuracy:  0.9833333492279053


In [ ]:
# Configuration related preprocessing step before mounting the drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate


KeyboardInterrupt: ignored

In [ ]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Save the model online
model.save('/content/drive/MyDrive/Colab Notebooks/covid19detector.h5')

In [ ]:
# =================== TESTING PART ======================== #

model1 = load_model('/content/drive/MyDrive/Colab Notebooks/covid19detector.h5')

In [ ]:
import os
train_generator.class_indices

In [ ]:
# Import few more necessary libraries.
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# Function to load and prepare the image in right shape
def load_image(filename):
  # Load the image
  img = image.load_img(filename, target_size=(224, 224))
  # Convert the image to array
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  # model = load_model('/content/drive/MyDrive/Colab Notebooks/covid19detector.h5') /loaded above/
  p = model.predict(img)
  
# Predict the output
print(p[0,0])

#Map apparel category with the numerical class
if p[0,0] == 0:
  final_result = "Covid Positive"
elif p[0,0] == 1:
  final_result = "Covid Negative"


In [ ]:
# Random testing 
# Covid +ve image returns 0 value
img = image.load_img('/content/CovidDataset/Val/Normal/NORMAL2-IM-0395-0001.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
p = model.predict(img)
print(p[0,0])

In [ ]:
# Covid -ve image returns 1 value
img = image.load_img('/content/CovidDataset/Val/Normal/NORMAL2-IM-0395-0001.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
p = model.predict(img)
# y_test.append(p[0,0])
# y_actual.append(1)
print(p[0,0])

if p[0,0] == 0.0:
  final_result = "Covid Positive"
elif p[0,0] == 1.0:
  final_result = "Covid Negative"
print(final_result)